In [183]:
import pandas as pd

In [184]:
from pyspark.sql import SparkSession
import pyspark
import os
# from dotenv import load_dotenv
# load_dotenv(override=True)
NESSIE_URI = "http://localhost:19120/api/v1"
WAREHOUSE = os.environ.get("WAREHOUSE")  
AWS_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.environ.get("AWS_SECRET_KEY")
AWS_S3_ENDPOINT = "http://localhost:9000"
conf = (
    pyspark.SparkConf()
        .setAppName("trade_pipeline")
        .set(
            "spark.jars.packages",
            ",".join([
                "org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1",
                "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0",
                "software.amazon.awssdk:bundle:2.17.178",
                "software.amazon.awssdk:url-connection-client:2.17.178"
            ])
        )
        .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
        .set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
        .set("spark.sql.catalog.nessie.uri", NESSIE_URI)
        .set("spark.sql.catalog.nessie.ref", "main")
        .set("spark.sql.catalog.nessie.authentication.type", "NONE")
        .set("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .set("spark.sql.catalog.nessie.warehouse", WAREHOUSE)
        .set("spark.sql.catalog.nessie.s3.endpoint", AWS_S3_ENDPOINT)
        .set("spark.sql.catalog.nessie.s3.path-style-access", "true")
        .set("spark.sql.catalog.nessie.client.timeout", "30s")
        # S3 Credentials
        .set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
        .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
        .set("spark.sql.defaultCatalog", "nessie")
        .set("spark.sql.catalog.demo", "")
)
spark = SparkSession.builder \
        .config(conf=conf) \
        .getOrCreate()

In [185]:
spark

25/09/03 20:12:21 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:593)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:592)
	at org.apache.spar

In [6]:
# query data from nessie
spark.sql("""select * from nessie.total_export_value_by_port limit 20""").show()

+----+-----+--------+--------------------+----+-------------------+-----------+
|YEAR|MONTH|CTY_CODE|            CTY_NAME|PORT|          PORT_NAME| ALL_VAL_MO|
+----+-----+--------+--------------------+----+-------------------+-----------+
|2015|    3|    null|                ASIA|   -|TOTAL FOR ALL PORTS|40349387606|
|2015|    3|       1|                OPEC|   -|TOTAL FOR ALL PORTS| 6370247872|
|2015|    3|       3|      EUROPEAN UNION|   -|TOTAL FOR ALL PORTS|24132189182|
|2015|    3|      14|PACIFIC RIM COUNT...|   -|TOTAL FOR ALL PORTS|32492696802|
|2015|    3|      17|            CAFTA-DR|   -|TOTAL FOR ALL PORTS| 2491510508|
|2015|    3|      20|       USMCA (NAFTA)|   -|TOTAL FOR ALL PORTS|45395687584|
|2015|    3|      21|TWENTY LATIN AMER...|   -|TOTAL FOR ALL PORTS|32162025588|
|2015|    3|      22|                OECD|   -|TOTAL FOR ALL PORTS|88373181240|
|2015|    3|      23|                NATO|   -|TOTAL FOR ALL PORTS|49463917409|
|2015|    3|      24|               LAFT

In [7]:
from minio import Minio
import io
def get_minio_client(minio_url, minio_access_key, minio_secret_key):
    try:
        minio_client = Minio(
            minio_url,
            access_key=minio_access_key,
            secret_key=minio_secret_key,
            secure=False,
        )
        return minio_client
    except Exception as e:
        raise

In [13]:
from dotenv import load_dotenv
load_dotenv(override=True)
minio_client = get_minio_client(
    "localhost:9000",
    os.getenv("MINIO_ACCESS_KEY"),
    os.getenv("MINIO_SECRET_KEY"),
)

In [11]:
os.getenv("MINIO_EXTERNAL_URL")

'minio:9000'

In [9]:
minio_client

In [139]:
objects_to_process = minio_client.list_objects(
            bucket_name=os.getenv("MINIO_BUCKET_NAME"), recursive=True
        )
minio_data = {}
for obj in objects_to_process:
    try:
        minio_response = minio_client.get_object(
            bucket_name=os.getenv("MINIO_BUCKET_NAME"),
            object_name=obj.object_name,
        )
        file_name = obj.object_name.split(".")[0]
        print(f"Processing file: {file_name}")
        data = minio_response.read()
        data = data.decode('utf-8', errors='replace')
        minio_response.close()
        minio_response.release_conn()
        minio_data[file_name] = pd.read_csv(io.StringIO(data))
    except Exception as e:
        print(f"Error processing object {obj.object_name}: {e}")


Processing file: china_tarriffs
Processing file: tariff_database_2025


/tmp/ipykernel_4158685/1984759914.py:17: DtypeWarning: Columns (19,61,66,85,86,95,107,111) have mixed types. Specify dtype option on import or set low_memory=False.
  minio_data[file_name] = pd.read_csv(io.StringIO(data))


In [142]:
china_tariff = minio_data.get("china_tarriffs")

In [140]:
tarrif_data = minio_data.get("tariff_database_2025")

In [141]:
tarrif_data.head()

,hts8,brief_description,quantity_1_code,quantity_2_code,wto_binding_code,mfn_text_rate,mfn_rate_type_code,mfn_ave,mfn_ad_val_rate,mfn_specific_rate,...,japan_indicator,japan_rate_type_code,japan_ad_val_rate,japan_specific_rate,japan_other_rate,usmca_indicator,usmca_rate_type_code,usmca_ad_val_rate,usmca_specific_rate,usmca_other_rate
0,1012100,Live purebred breeding horses,NO,NaN,B,Free,0,NaN,0.000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1012900,Live horses other than purebred breeding horses,NO,NaN,B,Free,0,NaN,0.000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1013000,Live asses,NO,NaN,B,6.8%,7,NaN,0.068,0.0,...,NaN,NaN,NaN,NaN,NaN,S,0.0,0.0,0.0,0.0
3,1019030,Mules and hinnies imported for immediate slaug...,NO,NaN,B,Free,0,NaN,0.000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1019040,Mules and hinnies not imported for immediate s...,NO,NaN,B,4.5%,7,NaN,0.045,0.0,...,NaN,NaN,NaN,NaN,NaN,S,0.0,0.0,0.0,0.0


In [21]:
tarrif_data.columns

Index(['hts8', 'brief_description', 'quantity_1_code', 'quantity_2_code',
       'wto_binding_code', 'mfn_text_rate', 'mfn_rate_type_code', 'mfn_ave',
       'mfn_ad_val_rate', 'mfn_specific_rate',
       ...
       'japan_indicator', 'japan_rate_type_code', 'japan_ad_val_rate',
       'japan_specific_rate', 'japan_other_rate', 'usmca_indicator',
       'usmca_rate_type_code', 'usmca_ad_val_rate', 'usmca_specific_rate',
       'usmca_other_rate'],
      dtype='object', length=122)

In [22]:
tarrif_data['hts8'].unique()

array([ 1012100,  1012900,  1013000, ..., 99990020, 99990084, 99999500])

In [48]:
print(tarrif_data.dtypes)

hts8                     int64
brief_description       object
quantity_1_code         object
quantity_2_code         object
wto_binding_code        object
                        ...   
usmca_ad_val_rate      float64
usmca_specific_rate    float64
usmca_other_rate       float64
hts8_str                object
HS_chapter               int64
Length: 124, dtype: object


In [70]:
tarrif_data[tarrif_data['hts8']==99030163][['hts8',
 'brief_description',
 'quantity_1_code',
 'quantity_2_code',
 'wto_binding_code',
 'mfn_text_rate',
 'mfn_rate_type_code',
 'mfn_ave',
 'mfn_ad_val_rate',
 'mfn_specific_rate',
 'mfn_other_rate',
 'col1_special_text',
 'col1_special_mod','begin_effect_date','end_effective_date']].dtypes


hts8                           int64
brief_description             object
quantity_1_code               object
quantity_2_code               object
wto_binding_code              object
mfn_text_rate                 object
mfn_rate_type_code            object
mfn_ave                      float64
mfn_ad_val_rate              float64
mfn_specific_rate            float64
mfn_other_rate               float64
col1_special_text             object
col1_special_mod             float64
begin_effect_date     datetime64[ns]
end_effective_date    datetime64[ns]
dtype: object

In [68]:
tarrif_data['begin_effect_date'] = pd.to_datetime(tarrif_data['begin_effect_date'], errors='coerce')
tarrif_data['end_effective_date'] = pd.to_datetime(tarrif_data['end_effective_date'], errors='coerce')

In [72]:
tarrif_data[['hts8',
 'brief_description',
 'quantity_1_code',
 'quantity_2_code',
 'wto_binding_code',
 'mfn_text_rate',
 'mfn_rate_type_code',
 'mfn_ave',
 'mfn_ad_val_rate',
 'mfn_specific_rate',
 'mfn_other_rate',
 'col1_special_text',
 'col1_special_mod','begin_effect_date','end_effective_date']].sort_values("begin_effect_date", ascending=False).head(50)

,hts8,brief_description,quantity_1_code,quantity_2_code,wto_binding_code,mfn_text_rate,mfn_rate_type_code,mfn_ave,mfn_ad_val_rate,mfn_specific_rate,mfn_other_rate,col1_special_text,col1_special_mod,begin_effect_date,end_effective_date
11900,99030163,Except for 9903.01.28-9903.01.33 and 9903.01.3...,NaN,NaN,U,The duty provided in the applicable subheading...,NaN,NaN,9999.999999,9999.999999,9999.999999,The duty provided in the applicable subheading...,NaN,2025-08-12,2050-12-31
12293,99039405,"Except for 9903.94.06 and 9903.94.32, automobi...",NaN,NaN,U,The duty provided in the applicable subheading...,NaN,NaN,9999.999999,9999.999999,9999.999999,The duty provided in the applicable subheading...,NaN,2025-06-30,2050-12-31
12295,99039431,Passenger vehicles that are products of the Un...,NaN,NaN,U,The duty provided in the applicable subheading...,NaN,NaN,9999.999999,9999.999999,9999.999999,No change.,NaN,2025-06-30,2050-12-31
12296,99039432,Parts of passenger vehicles and light trucks o...,NaN,NaN,U,10%,NaN,NaN,9999.999999,9999.999999,9999.999999,10%,NaN,2025-06-30,2050-12-31
12297,99039601,"Articles of civil aircraft; their engines, par...",NaN,NaN,U,The duty provided in the applicable subheading,NaN,NaN,9999.999999,9999.999999,9999.999999,The duty provided in the applicable subheading,NaN,2025-06-30,2050-12-31
11867,99030125,"Articles the product of any country, except de...",NaN,NaN,U,The duty provided in the applicable subheading...,NaN,NaN,9999.999999,9999.999999,9999.999999,The duty provided in the applicable subheading...,NaN,2025-06-30,2050-12-31
12288,99039401,"Except for 9903.94.02, 9903.94.03, 9903.94.04 ...",NaN,NaN,U,The duty provided in the applicable subheading...,NaN,NaN,9999.999999,9999.999999,9999.999999,The duty provided in the applicable subheading...,NaN,2025-06-30,2050-12-31
12195,99038514,"Derivative aluminum products of the UK, provid...",NaN,NaN,U,The duty provided in the applicable subheading...,NaN,NaN,9999.999999,9999.999999,9999.999999,The duty provided in the applicable subheading...,NaN,2025-06-04,2050-12-31
12194,99038513,Derivative aluminum products of the UK provide...,NaN,NaN,U,The duty provided in the applicable subheading...,NaN,NaN,9999.999999,9999.999999,9999.999999,The duty provided in the applicable subheading...,NaN,2025-06-04,2050-12-31
12193,99038512,Products of aluminum of the UK provided for in...,NaN,NaN,U,The duty provided in the applicable subheading...,NaN,NaN,9999.999999,9999.999999,9999.999999,The duty provided in the applicable subheading...,NaN,2025-06-04,2050-12-31


In [75]:
tarriff_temporary = tarrif_data[tarrif_data['hts8_str'].str.startswith('9903')]

In [104]:
tarriff_temporary[[
    'hts8_str',
    'brief_description',
    # 'quantity_1_code',
    # 'quantity_2_code',
    'wto_binding_code',
    'mfn_text_rate',
    # 'mfn_rate_type_code',
    # 'mfn_ave',
    # 'mfn_ad_val_rate',
    # 'mfn_specific_rate',
    # 'mfn_other_rate',
    'col1_special_text',
    'col1_special_mod','begin_effect_date','end_effective_date'
]]

,hts8_str,brief_description,wto_binding_code,mfn_text_rate,col1_special_text,col1_special_mod,begin_effect_date,end_effective_date
11846,99030101,"Articles the product of Mexico, as provided fo...",U,The duty provided in the applicable subheading...,The duty provided in the applicable subheading...,NaN,2025-03-04,2025-03-06
11847,99030101,"Articles the product of Mexico, as provided fo...",U,The duty provided in the applicable subheading...,The duty provided in the applicable subheading...,NaN,2025-03-07,2050-12-31
11848,99030102,Articles the product of Mexico that are donati...,U,The duty provided in the applicable subheading,The duty provided in the applicable subheading,NaN,2025-03-04,2050-12-31
11849,99030103,Articles the product of Mexico that are inform...,U,The duty provided in the applicable subheading,The duty provided in the applicable subheading,NaN,2025-03-04,2050-12-31
11850,99030104,Articles that are entered free of duty under G...,U,No change,The duty provided in the applicable subheading,NaN,2025-03-07,2050-12-31
...,...,...,...,...,...,...,...,...
12293,99039405,"Except for 9903.94.06 and 9903.94.32, automobi...",U,The duty provided in the applicable subheading...,The duty provided in the applicable subheading...,NaN,2025-06-30,2050-12-31
12294,99039406,Articles provided for in US Note 33(h) to this...,U,The duty provided in the applicable subheading,The duty provided in the applicable subheading,NaN,2025-05-03,2050-12-31
12295,99039431,Passenger vehicles that are products of the Un...,U,The duty provided in the applicable subheading...,No change.,NaN,2025-06-30,2050-12-31
12296,99039432,Parts of passenger vehicles and light trucks o...,U,10%,10%,NaN,2025-06-30,2050-12-31


In [105]:
tarriff_temporary['col1_special_text'].unique()

array(['The duty provided in the applicable subheading + 25%',
       'The duty provided in the applicable subheading', 'No change',
       'The duty provided in the applicable subheading + 10%',
       'The duty provided in the applicable subheading + 20%',
       'The duty provided in the applicable subheading + 11%',
       'The duty provided in the applicable subheading + 13%',
       'The duty provided in the applicable subheading + 14%',
       'The duty provided in the applicable subheading + 15%',
       'The duty provided in the applicable subheading + 16%',
       'The duty provided in the applicable subheading + 17%',
       'The duty provided in the applicable subheading + 18%',
       'The duty provided in the applicable subheading + 21%',
       'The duty provided in the applicable subheading + 22%',
       'The duty provided in the applicable subheading + 24%',
       'The duty provided in the applicable subheading + 26%',
       'The duty provided in the applicable subh

In [98]:
tarriff_temporary['mfn_text_rate'].unique()

array(['The duty provided in the applicable subheading + 25%',
       'The duty provided in the applicable subheading', 'No change',
       'The duty provided in the applicable subheading + 10%',
       'The duty provided in the applicable subheading + 20%',
       'The duty provided in the applicable subheading + 11%',
       'The duty provided in the applicable subheading + 13%',
       'The duty provided in the applicable subheading + 14%',
       'The duty provided in the applicable subheading + 15%',
       'The duty provided in the applicable subheading + 16%',
       'The duty provided in the applicable subheading + 17%',
       'The duty provided in the applicable subheading + 18%',
       'The duty provided in the applicable subheading + 21%',
       'The duty provided in the applicable subheading + 22%',
       'The duty provided in the applicable subheading + 24%',
       'The duty provided in the applicable subheading + 26%',
       'The duty provided in the applicable subh

In [29]:
# find agri hts8 codes
tarrif_data['hts8_str'] = tarrif_data['hts8'].astype(str).str.zfill(8)

In [30]:
tarrif_data['HS_chapter'] = tarrif_data['hts8_str'].str[:2].astype(int)

In [32]:
agri_tarrif_data = tarrif_data[(tarrif_data['HS_chapter'] >= 1) & (tarrif_data['HS_chapter'] <= 24)]
agri_tarrif_data.shape

(2162, 124)

In [34]:
agri_tarrif_data.columns.to_list()

['hts8',
 'brief_description',
 'quantity_1_code',
 'quantity_2_code',
 'wto_binding_code',
 'mfn_text_rate',
 'mfn_rate_type_code',
 'mfn_ave',
 'mfn_ad_val_rate',
 'mfn_specific_rate',
 'mfn_other_rate',
 'col1_special_text',
 'col1_special_mod',
 'gsp_indicator',
 'gsp_ctry_excluded',
 'apta_indicator',
 'civil_air_indicator',
 'nafta_canada_ind',
 'nafta_mexico_ind',
 'mexico_rate_type_code',
 'mexico_ad_val_rate',
 'mexico_specific_rate',
 'cbi_indicator',
 'cbi_ad_val_rate',
 'cbi_specific_rate',
 'agoa_indicator',
 'cbtpa_indicator',
 'cbtpa_rate_type_code',
 'cbtpa_ad_val_rate',
 'cbtpa_specific_rate',
 'israel_fta_indicator',
 'atpa_indicator',
 'atpa_ad_val_rate',
 'atpa_specific_rate',
 'atpdea_indicator',
 'jordan_indicator',
 'jordan_rate_type_code',
 'jordan_ad_val_rate',
 'jordan_specific_rate',
 'jordan_other_rate',
 'singapore_indicator',
 'singapore_rate_type_code',
 'singapore_ad_val_rate',
 'singapore_specific_rate',
 'singapore_other_rate',
 'chile_indicator',
 'ch

In [36]:
subset_rice = agri_tarrif_data[agri_tarrif_data['hts8_str'].str.startswith('1006')]
subset_rice

,hts8,brief_description,quantity_1_code,quantity_2_code,wto_binding_code,mfn_text_rate,mfn_rate_type_code,mfn_ave,mfn_ad_val_rate,mfn_specific_rate,...,japan_ad_val_rate,japan_specific_rate,japan_other_rate,usmca_indicator,usmca_rate_type_code,usmca_ad_val_rate,usmca_specific_rate,usmca_other_rate,hts8_str,HS_chapter
1134,10061000,Rice in the husk (paddy or rough),KG,NaN,B,1.8 cents/kg,1,NaN,0.000,0.0180,...,NaN,NaN,NaN,S,0.0,0.0,0.0,0.0,10061000,10
1135,10062020,"Basmati rice, husked",KG,NaN,B,0.83 cents/kg,1,NaN,0.000,0.0083,...,NaN,NaN,NaN,S,0.0,0.0,0.0,0.0,10062020,10
1136,10062040,"Husked (brown) rice, other than Basmati",KG,NaN,B,2.1 cents/kg,1,NaN,0.000,0.0210,...,NaN,NaN,NaN,S,0.0,0.0,0.0,0.0,10062040,10
1137,10063010,"Rice semi-milled or wholly milled, whether or ...",KG,NaN,B,11.2%,7,NaN,0.112,0.0000,...,NaN,NaN,NaN,S,0.0,0.0,0.0,0.0,10063010,10
1138,10063090,"Rice semi-milled or wholly milled, whether or ...",KG,NaN,B,1.4 cents/kg,1,NaN,0.000,0.0140,...,NaN,NaN,NaN,S,0.0,0.0,0.0,0.0,10063090,10
1139,10064000,Broken rice,KG,NaN,B,0.44 cents/kg,1,NaN,0.000,0.0044,...,NaN,NaN,NaN,S,0.0,0.0,0.0,0.0,10064000,10


In [43]:
japan_tarrif_on_rice = subset_rice[['hts8',
 'brief_description',
 'quantity_1_code',
 'quantity_2_code',
 'wto_binding_code',
 'mfn_text_rate',
 'mfn_rate_type_code',
 'mfn_ave',
 'mfn_ad_val_rate',
 'mfn_specific_rate',
 'mfn_other_rate',
 'col1_special_text',
 'col1_special_mod',
 'gsp_indicator',
 'gsp_ctry_excluded',
 'apta_indicator',
 'civil_air_indicator',
"begin_effect_date", "end_effective_date", "japan_indicator","japan_ad_val_rate", "japan_specific_rate"]]
japan_tarrif_on_rice

,hts8,brief_description,quantity_1_code,quantity_2_code,wto_binding_code,mfn_text_rate,mfn_rate_type_code,mfn_ave,mfn_ad_val_rate,mfn_specific_rate,...,mexico_ad_val_rate,mexico_specific_rate,cbi_indicator,cbi_ad_val_rate,cbi_specific_rate,begin_effect_date,end_effective_date,japan_indicator,japan_ad_val_rate,japan_specific_rate
1134,10061000,Rice in the husk (paddy or rough),KG,NaN,B,1.8 cents/kg,1,NaN,0.000,0.0180,...,0.0,0.0,E,0.0,0.0,07/01/2020,12/31/2050,NaN,NaN,NaN
1135,10062020,"Basmati rice, husked",KG,NaN,B,0.83 cents/kg,1,NaN,0.000,0.0083,...,0.0,0.0,E,0.0,0.0,07/01/2020,12/31/2050,NaN,NaN,NaN
1136,10062040,"Husked (brown) rice, other than Basmati",KG,NaN,B,2.1 cents/kg,1,NaN,0.000,0.0210,...,0.0,0.0,E,0.0,0.0,07/01/2020,12/31/2050,NaN,NaN,NaN
1137,10063010,"Rice semi-milled or wholly milled, whether or ...",KG,NaN,B,11.2%,7,NaN,0.112,0.0000,...,0.0,0.0,E,0.0,0.0,01/01/2024,12/31/2050,NaN,NaN,NaN
1138,10063090,"Rice semi-milled or wholly milled, whether or ...",KG,NaN,B,1.4 cents/kg,1,NaN,0.000,0.0140,...,0.0,0.0,E,0.0,0.0,07/01/2020,12/31/2050,NaN,NaN,NaN
1139,10064000,Broken rice,KG,NaN,B,0.44 cents/kg,1,NaN,0.000,0.0044,...,0.0,0.0,E,0.0,0.0,07/01/2020,12/31/2050,NaN,NaN,NaN


In [133]:
china_tarriffs = pd.read_csv("../data/china_tarriffs.csv")

In [143]:
china_tarriffs.tail()

,Harmonized Tariff Schedule 8‐digit Subheading
9705.29.00,9903.88.15
9705.31.00,9903.88.15
9705.39.00,9903.88.15
9706.10.00,9903.88.15
9706.90.00,9903.88.15


In [216]:
import pandas as pd
import requests
import xmltodict

# Define product codes and years
product_list = ["940370"]  # e.g. ["100199", "100190"]
years_list = list(range(1988, 2021))

# Get country codes classified as reporters
countries_url = "http://wits.worldbank.org/API/V1/wits/datasource/trn/country/ALL"
countries_xml = requests.get(countries_url).content
countries_dict = xmltodict.parse(countries_xml)

In [225]:
countries_dict['wits:datasource'].keys()

dict_keys(['@datasourcecode', '@datasourcename', '@language', '@total', '@xmlns:wits', 'wits:countries'])

In [222]:
country_details = countries_dict['wits:datasource']['wits:countries']['wits:country']

reporter_list = [
    c['@countrycode']
    for c in country_details
    if c.get('@isreporter') == "1"
]
partner_list = [
    c['@countrycode']
    for c in country_details
    if c.get('@isreporter') in ["1", "0"]
]

In [226]:
# get nominclature

url = "http://wits.worldbank.org/API/V1/wits/datasource/trn/nomenclature/ALL"
response = requests.get(url)
print(response.status_code)

200


In [230]:
xml_response = response.content
nomenclature_dict = xmltodict.parse(xml_response)

In [236]:
nomenclature_dict['wits:datasource']['wits:nomenclatures']['wits:nomenclature']

[{'@nomenclaturecode': 'H0',
  '@description': 'Harmonized System 1988/92',
  '#text': 'HS 1988/92'},
 {'@nomenclaturecode': 'H1',
  '@description': 'Harmonized System 1996',
  '#text': 'HS 1996'},
 {'@nomenclaturecode': 'H2',
  '@description': 'Harmonized System 2002',
  '#text': 'HS 2002'},
 {'@nomenclaturecode': 'H3',
  '@description': 'Harmonized System 2007',
  '#text': 'HS 2007'},
 {'@nomenclaturecode': 'H4',
  '@description': 'Harmonized System 2012',
  '#text': 'HS 2012'},
 {'@nomenclaturecode': 'H5',
  '@description': 'Harmonized System 2017',
  '#text': 'HS 2017'},
 {'@nomenclaturecode': 'H6',
  '@description': 'Harmonized System 2022',
  '#text': 'HS 2022'}]

In [498]:
# get product 
url = "http://wits.worldbank.org/API/V1/wits/datasource/trn/product/ALL"
response = requests.get(url)
print(response.status_code)
products_dict = xmltodict.parse(response.content)

200


In [ ]:
pd.json_normalize(products_dict['wits:datasource']['wits:products']['wits:product']).rename(columns=lambda x: x.replace('@', '')).head()

,productcode,isgroup,nomenclaturecode,grouptype,wits:productdescription,wits:notes
0,010110,No,HS,N/A,010110 -- (2002-2011) - Pure-bred breeding ani...,None
1,010111,No,HS,N/A,010111 -- (-2001) -- Pure-bred breeding animals,None
2,010119,No,HS,N/A,010119 -- (-2001) -- Other,None
3,010120,No,HS,N/A,"010120 -- (-2001) - Asses, mules and hinnies",None
4,010121,No,HS,N/A,010121 -- (2012-) -- Pure-bred breeding animals,None


In [503]:
# data availability
url = "http://wits.worldbank.org/API/V1/wits/datasource/trn/dataavailability/country/ALL/year/ALL"
response = requests.get(url)
print(response.status_code)
data_availability_dict = xmltodict.parse(response.content)

200


In [504]:
data_availability_dict['wits:datasource']['wits:dataavailability']['wits:reporter'][0]

{'@countrycode': '012',
 '@iso3Code': 'DZA',
 '@isgroup': 'No',
 '@grouptype': 'N/A',
 'wits:name': 'Algeria',
 'wits:year': '1993',
 'wits:reporternernomenclature': {'@reporternernomenclaturecode': 'H0',
  '#text': 'Harmonized System 1988/92'},
 'wits:numberofpreferentialagreement': '1',
 'wits:partnerlist': '000;N12;',
 'wits:isspecificdutyexpressionestimatedavailable': 'No',
 'wits:notes': None,
 'wits:lastupdateddate': '2014/08/04'}

In [313]:
url = "https://wits.worldbank.org/API/V1/SDMX/V21/datasource/TRN/reporter/040/partner/all/product/940370/year/2020/datatype/reported?format=JSON"
response = requests.get(url)
print(response.status_code)
data = response.json()


200


In [314]:
data

{'header': {'id': 'DF_WITS_Tariff_TRAINS',
  'prepared': '2025-09-04T14:27:32',
  'test': False,
  'sender': {'id': 'WBG_WITS', 'name': 'World Bank Group - WITS'}},
 'dataSets': [{'action': 'Information',
   'attributes': [0],
   'series': {'0:0:0:0:0': {'attributes': [],
     'annotations': [],
     'observations': {'0': [0, 0, None, 0, 0, 0, 0, 0, 0, 0, 0, 0]}}}}],
 'structure': {'name': 'WITS - UNCTAD TRAINS Tariff Data',
  'description': 'Data flow to access WITS - UNCTAD TRAINS Preferential and most-favored-nation (MFN) tariff rates',
  'dimensions': {'dataset': [],
   'series': [{'id': 'FREQ',
     'name': 'Freq',
     'keyPosition': 0,
     'role': None,
     'values': [{'id': 'A', 'name': 'Annual'}]},
    {'id': 'REPORTER',
     'name': 'Reporter_ISO_N',
     'keyPosition': 1,
     'role': None,
     'values': [{'id': '040', 'name': 'Austria'}]},
    {'id': 'PARTNER',
     'name': 'Partner',
     'keyPosition': 2,
     'role': None,
     'values': [{'id': '000', 'name': 'World'

In [364]:
endpoint = "https://wits.worldbank.org/API/V1/SDMX/V21/rest" 

In [365]:
path = '/'.join([endpoint, "dataflow"])
print(path)
try:
    response = requests.get(path)
    response_dict = xmltodict.parse(response.text)
    print(response_dict)    
except Exception as e:
    print(f"Error fetching dataflow: {e}")
    raise e


https://wits.worldbank.org/API/V1/SDMX/V21/rest/dataflow
{'Structure': {'@xmlns': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message', 'Header': {'ID': 'IDREF26', 'Test': 'false', 'Prepared': '2025-09-04T16:01:16.2507882-04:00', 'Sender': {'@id': 'Unknown'}, 'Receiver': {'@id': 'Unknown'}}, 'Structures': {'Dataflows': {'@xmlns': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure', 'Dataflow': [{'@id': 'DF_WITS_Tariff_TRAINS', '@urn': 'urn:sdmx:org.sdmx.infomodel.datastructure.Dataflow=WBG_WITS:DF_WITS_Tariff_TRAINS(1.1)', '@agencyID': 'WBG_WITS', '@version': '1.1', '@isFinal': 'false', 'Name': {'@xml:lang': 'en', '@xmlns': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common', '#text': 'WITS - UNCTAD TRAINS Tariff Data'}, 'Description': {'@xml:lang': 'en', '@xmlns': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common', '#text': 'Data flow to access WITS - UNCTAD TRAINS Preferential and most-favored-nation (MFN) tariff rates'}, 'Structure': {'Ref': {'@id': '

In [375]:
dataflows = pd.json_normalize(response_dict['Structure']['Structures']['Dataflows']['Dataflow'])

In [376]:
dataflows = dataflows[(dataflows['Name.@xml:lang'] == 'en') &
                        (dataflows['Description.@xml:lang'] == 'en')
                    ]
dataflows = dataflows[['@id', '@agencyID', '@version', '@isFinal', 'Description.#text', 'Structure.Ref.@id']]


In [378]:
dataflows = dataflows.rename(columns={
    '@id': 'id',
    '@agencyID': 'agencyID',
    '@version': 'version',
    '@isFinal':'isFinal',
    'Description.#text': 'description',
    'Structure.Ref.@id': 'datastructure'
})

display(dataflows[['id', 'datastructure', 'description']])

,id,datastructure,description
0,DF_WITS_Tariff_TRAINS,TARIFF_TRAINS,Data flow to access WITS - UNCTAD TRAINS Prefe...
1,DF_WITS_TradeStats_Development,TRADESTATS,"Development indicators such as GDP, GNI per ca..."
2,DF_WITS_TradeStats_Tariff,TRADESTATS,Tariff information like number of trade agreem...
3,DF_WITS_TradeStats_Trade,TRADESTATS,"Trade data such as total exports, number or pr..."


In [379]:
dataset_id = 'DF_WITS_TradeStats_Trade'
datastructure_id = 'TRADESTATS' 

In [ ]:
path = '/'.join([endpoint, 'datastructure/WBG_WITS', datastructure_id])
try:
    response = requests.get(path)
    response_dict = xmltodict.parse(response.text)
    print(response_dict)
except Exception as e:
    print(f"Error fetching datastructure: {e}")
    raise e

{'Structure': {'@xmlns': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message', 'Header': {'ID': 'IDREF27', 'Test': 'false', 'Prepared': '2025-09-04T16:10:51.5502873-04:00', 'Sender': {'@id': 'Unknown'}, 'Receiver': {'@id': 'Unknown'}}, 'Structures': {'DataStructures': {'@xmlns': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure', 'DataStructure': {'@id': 'TRADESTATS', '@urn': 'urn:sdmx:org.sdmx.infomodel.datastructure.DataStructure=WBG_WITS:TRADESTATS(1.0)', '@agencyID': 'WBG_WITS', '@version': '1.0', '@isFinal': 'true', 'Name': {'@xml:lang': 'en', '@xmlns': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common', '#text': 'Country Trade Statistics'}, 'DataStructureComponents': {'DimensionList': {'@id': 'DimensionDescriptor', '@urn': 'urn:sdmx:org.sdmx.infomodel.datastructure.DimensionDescriptor=WBG_WITS:TRADESTATS(1.0).DimensionDescriptor', 'Dimension': [{'@id': 'FREQ', '@urn': 'urn:sdmx:org.sdmx.infomodel.datastructure.Dimension=WBG_WITS:TRADESTATS(1.0).FREQ', '@

In [385]:
datastructure = response_dict['Structure']['Structures']['DataStructures']['DataStructure']

In [398]:
datastructure_meta = {
    'id': datastructure['@id'],
    'agencyID': datastructure['@agencyID'],
    'version': datastructure['@version'],
    'isFinal': datastructure['@isFinal'],
    'name': datastructure['Name']['#text']
}

dimensions = pd.json_normalize(datastructure['DataStructureComponents']['DimensionList']['Dimension'])[['@id', '@position', 'ConceptIdentity.Ref.@maintainableParentID', 'LocalRepresentation.Enumeration.Ref.@id']]
dimensions = dimensions.rename(columns={
    '@id': 'id',
    '@position': 'position',
    'ConceptIdentity.Ref.@maintainableParentID': 'conceptscheme',
    'LocalRepresentation.Enumeration.Ref.@id': 'codelist'
})


In [443]:
dimensions

,id,position,conceptscheme,codelist
0,FREQ,1,TRADESTATS_CONCEPTS,CL_TS_FREQ_WITS
1,REPORTER,2,TRADESTATS_CONCEPTS,CL_TS_COUNTRY_WITS
2,PARTNER,3,TRADESTATS_CONCEPTS,CL_TS_COUNTRY_WITS
3,PRODUCTCODE,5,TRADESTATS_CONCEPTS,CL_TS_PRODUCTCODE_WITS
4,INDICATOR,6,TRADESTATS_CONCEPTS,CL_TS_INDICATOR_WITS


In [400]:
dimension_list = dimensions['id'].values.tolist()
query = {k: None for k in dimension_list}

def getCodelist(codelist):
    codelist_meta = {
                'id':codelist['@id'],
                'agencyID': codelist['@agencyID'],
                'version': codelist['@version'],
                'isFinal': codelist['@isFinal'],
                'name': codelist['Name']['#text']
            }
    codelist_codes = codelist['Code']

    if type(codelist_codes) == list:
        codelist_code = [{'id': code['@id'],
                        'name': code['Name']['#text'],
                        'language': code['Name']['@xml:lang']} for code in codelist_codes]
    else:
        codelist_code = [{'id': codelist_codes['@id'],
        'name': codelist_codes['Name']['#text'],
        'language': codelist_codes['Name']['@xml:lang']}]

    return {
        'header': pd.DataFrame([codelist_meta]),
        'codes': pd.DataFrame(codelist_code)
    }

In [403]:
codelist = 'CL_TS_FREQ_WITS'
path = '/'.join([endpoint, 'codelist/WBG_WITS', codelist])

response = requests.get(path)
if response.status_code == 200:
   response_dict = xmltodict.parse(response.text)['Structure']
   codelists = response_dict['Structures']['Codelists']['Codelist']
   codelist = getCodelist(codelists)['codes']

display(codelist)

,id,name,language
0,A,Annual,en


In [404]:
query['FREQ'] = 'A'

In [520]:
codelist = 'CL_TS_COUNTRY_WITS'
path = '/'.join([endpoint, 'codelist/WBG_WITS', codelist])

response = requests.get(path)
if response.status_code == 200:
   response_dict = xmltodict.parse(response.text)['Structure']
   codelists = response_dict['Structures']['Codelists']['Codelist']
   codelist = getCodelist(codelists)['codes']

filtered_codes = codelist[codelist['name'].isin(['Singapore', 'Malaysia', 'Indonesia'])]
display(filtered_codes)

,id,name,language
110,IDN,Indonesia,en
163,MYS,Malaysia,en
205,SGP,Singapore,en


In [526]:
codelist[codelist['name'].str.contains('United States')]

,id,name,language
244,UMI,United States Minor Outlying I,en
247,USA,United States,en


In [406]:
query['REPORTER'] = 'IDN'
query['PARTNER'] = 'SGP+MYS'

In [453]:
codelist = 'CL_TS_PRODUCTCODE_WITS'
path = '/'.join([endpoint, 'codelist/WBG_WITS', codelist])
print(path)
response = requests.get(path)
print(response.status_code)
if response.status_code == 200:
   response_dict = xmltodict.parse(response.text)['Structure']
   codelists = response_dict['Structures']['Codelists']['Codelist']
   codelist = getCodelist(codelists)['codes']
   header = getCodelist(codelists)['header']
display(codelist)
display(header)

https://wits.worldbank.org/API/V1/SDMX/V21/rest/codelist/WBG_WITS/CL_TS_PRODUCTCODE_WITS
200


,id,name,language
0,999999,Not Applicable,en
1,01-05_Animal,Animal,en
2,06-15_Vegetable,Vegetable,en
3,16-24_FoodProd,Food Products,en
4,25-26_Minerals,Minerals,en
5,27-27_Fuels,Fuels,en
6,28-38_Chemicals,Chemicals,en
7,39-40_PlastiRub,Plastic or Rubber,en
8,41-43_HidesSkin,Hides and Skins,en
9,44-49_Wood,Wood,en


,id,agencyID,version,isFinal,name
0,CL_TS_PRODUCTCODE_WITS,WBG_WITS,1.0,true,Product Code


In [513]:
query['PRODUCTCODE'] = "AgrRaw"

In [514]:
query

{'FREQ': 'A',
 'REPORTER': 'IDN',
 'PARTNER': 'SGP+MYS',
 'PRODUCTCODE': 'AgrRaw',
 'INDICATOR': 'XPRT-TRD-VL'}

In [515]:
codelist = 'CL_TS_INDICATOR_WITS'
path = '/'.join([endpoint, 'codelist/WBG_WITS', codelist])
response = requests.get(path)
if response.status_code == 200:
   response_dict = xmltodict.parse(response.text)['Structure']
   codelists = response_dict['Structures']['Codelists']['Codelist']
   codelist = getCodelist(codelists)['codes']

In [516]:
codelist[codelist['name'].str.contains('Export')]

,id,name,language
33,NMBR-XPRT-HS6-PRDCT,Export No Of traded HS6 digit Products,en
36,XPRT-PRDCT-SHR,Export Product Share (%),en
37,XPRT-PRTNR-SHR,Export Partner Share (%),en
38,XPRT-SHR-TTL-PRDCT,Export Share in Total Products (%),en
39,XPRT-TRD-VL,Export Trade Value (US$ Thousand),en
56,BX-GSR-GNFS-CD,"Exports of goods and services (BoP, current US$)",en
59,BX-GSR-TOTL-CD,"Exports of goods, services and primary income ...",en
78,NE-EXP-GNFS-CD,Exports of goods and services (current US$),en
79,NE-EXP-GNFS-KD,Exports of goods and services (constant 2005 US$),en
80,NE-EXP-GNFS-KD-ZG,Exports of goods and services (annual % growth),en


In [414]:
query['INDICATOR'] = 'XPRT-TRD-VL'

In [517]:
query_string = '.'.join([value for value in query.values()])

In [518]:
query_string

'A.IDN.SGP+MYS.AgrRaw.XPRT-TRD-VL'

In [474]:
period_parameter = '?startperiod=2010&endperiod=2015'

In [475]:
path = '/'.join([endpoint, 'data', dataset_id, query_string, period_parameter])
print(path)
response = requests.get(path)

try:
    response = requests.get(path)
    response_dict = xmltodict.parse(response.text)
    print(response_dict)
except Exception as e:
    print(f"Error fetching data: {e}")
    raise e

https://wits.worldbank.org/API/V1/SDMX/V21/rest/data/DF_WITS_TradeStats_Trade/A.IDN.SGP+MYS.Fuels.XPRT-TRD-VL/?startperiod=2010&endperiod=2015
{'message:GenericData': {'@xmlns:footer': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message/footer', '@xmlns:generic': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic', '@xmlns:message': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message', '@xmlns:common': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xmlns:xml': 'http://www.w3.org/XML/1998/namespace', 'message:Header': {'message:ID': 'DF_WITS_TradeStats_Trade', 'message:Test': 'false', 'message:Prepared': '2025-09-04T17:44:45', 'message:Sender': {'@id': 'WBG_WITS', 'common:Name': {'@xml:lang': 'en', '#text': 'World Bank Group - WITS'}, 'message:Contact': {'common:Name': {'@xml:lang': 'en', '#text': 'WITS Admin'}, 'message:Department': {'@xml:lang': 'en', '#text': 'DECDG'}, 'message:Role':

In [519]:
path = '/'.join([endpoint, 'data', dataset_id, query_string, period_parameter])
print(path)
response = requests.get(path)
if response.status_code == 200:
   response_dict = xmltodict.parse(response.text)
   response_series = response_dict['message:GenericData']['message:DataSet']['generic:Series']
   series_list = [response_series] if type(response_series) == dict else response_series
   for i, series in enumerate(series_list):
       series_key = series['generic:SeriesKey']
       series_key_values_raw = [series_key['generic:Value']] if type(series_key['generic:Value']) == dict else series_key['generic:Value']
       series_key_values = {value['@id']:value['@value'] for value in series_key_values_raw}

       series_obs = series['generic:Obs']
       series_obs_raw = [series['generic:Obs']] if type(series['generic:Obs']) == dict else series['generic:Obs']
       for j, obs in enumerate(series_obs_raw):
           obs_dimensions_raw = [obs['generic:ObsDimension']] if type(obs['generic:ObsDimension']) == dict else obs['generic:ObsDimension'] 
           obs_dimensions = {value['@id']: value['@value'] for value in obs_dimensions_raw}

           obs_attributes = obs['generic:Attributes']['generic:Value']
           obs_attributes = [obs_attributes] if type(obs_attributes) == dict else obs_attributes
           obs_attributes = {value['@id']: value['@value'] for value in obs_attributes}

           obs_value = obs['generic:ObsValue']
           obs_value = {'OBS_VALUE': float(obs_value['@value'])}
          
           observation = [{**series_key_values, **obs_dimensions, **obs_attributes, **obs_value}]
           observations = observation if j == 0 else observations + observation
       series_observations = observations if i == 0 else series_observations + observations

   data = pd.DataFrame(series_observations)
display(data)

https://wits.worldbank.org/API/V1/SDMX/V21/rest/data/DF_WITS_TradeStats_Trade/A.IDN.SGP+MYS.AgrRaw.XPRT-TRD-VL/?startperiod=2010&endperiod=2015


,FREQ,REPORTER,PARTNER,PRODUCTCODE,INDICATOR,TIME_PERIOD,DATASOURCE,OBS_VALUE
0,A,IDN,MYS,AgrRaw,XPRT-TRD-VL,2010,WITS-CMT,49864.549
1,A,IDN,MYS,AgrRaw,XPRT-TRD-VL,2011,WITS-CMT,48969.180
2,A,IDN,MYS,AgrRaw,XPRT-TRD-VL,2012,WITS-CMT,60452.705
3,A,IDN,MYS,AgrRaw,XPRT-TRD-VL,2013,WITS-CMT,52625.428
4,A,IDN,MYS,AgrRaw,XPRT-TRD-VL,2014,WITS-CMT,43469.375
5,A,IDN,MYS,AgrRaw,XPRT-TRD-VL,2015,WITS-CMT,37050.617
6,A,IDN,SGP,AgrRaw,XPRT-TRD-VL,2010,WITS-CMT,387193.909
7,A,IDN,SGP,AgrRaw,XPRT-TRD-VL,2011,WITS-CMT,512514.827
8,A,IDN,SGP,AgrRaw,XPRT-TRD-VL,2012,WITS-CMT,232966.538
9,A,IDN,SGP,AgrRaw,XPRT-TRD-VL,2013,WITS-CMT,95175.765


In [ ]:
series_key_values_raw

[{'@id': 'FREQ', '@value': 'A'},
 {'@id': 'REPORTER', '@value': 'IDN'},
 {'@id': 'PARTNER', '@value': 'MYS'},
 {'@id': 'PRODUCTCODE', '@value': 'Fuels'},
 {'@id': 'INDICATOR', '@value': 'XPRT-TRD-VL'}]

In [332]:
products = "020110"
reporter = "840"
year = 2000
url = f"http://wits.worldbank.org/API/V1/SDMX/V21/rest/data/DF_WITS_Tariff_TRAINS/.{reporter}.000.{products}.reported/?startperiod={year}&endperiod={year}"

headers = {
    "Accept": "application/vnd.sdmx.data+json;version=1.0.0-wd"
}

resp = requests.get(url)#, headers=headers)
print(resp.status_code)
print(url)

404
http://wits.worldbank.org/API/V1/SDMX/V21/rest/data/DF_WITS_Tariff_TRAINS/.840.000.020110.reported/?startperiod=2000&endperiod=2000


In [334]:
import pandasdmx as sdmx

In [256]:
data['header']

{'id': 'DF_WITS_Tariff_TRAINS',
 'prepared': '2025-09-04T12:22:33',
 'test': False,
 'sender': {'id': 'WBG_WITS', 'name': 'World Bank Group - WITS'}}

In [258]:
data['dataSets'][0]

{'action': 'Information',
 'attributes': [0],
 'series': {'0:0:0:0:0': {'attributes': [],
   'annotations': [],
   'observations': {'0': [26.3999996185303,
     0,
     None,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0]}}}}

In [245]:
data['structure']

{'name': 'WITS - UNCTAD TRAINS Tariff Data',
 'description': 'Data flow to access WITS - UNCTAD TRAINS Preferential and most-favored-nation (MFN) tariff rates',
 'dimensions': {'dataset': [],
  'series': [{'id': 'FREQ',
    'name': 'Freq',
    'keyPosition': 0,
    'role': None,
    'values': [{'id': 'A', 'name': 'Annual'}]},
   {'id': 'REPORTER',
    'name': 'Reporter_ISO_N',
    'keyPosition': 1,
    'role': None,
    'values': [{'id': '840', 'name': 'United States'}]},
   {'id': 'PARTNER',
    'name': 'Partner',
    'keyPosition': 2,
    'role': None,
    'values': [{'id': '000', 'name': 'World'}]},
   {'id': 'PRODUCTCODE',
    'name': 'ProductCode',
    'keyPosition': 3,
    'role': None,
    'values': [{'id': '020110', 'name': '- Carcasses and half-carcasses'}]},
   {'id': 'DATATYPE',
    'name': 'DataType',
    'keyPosition': 4,
    'role': None,
    'values': [{'id': 'Reported',
      'name': 'Reported Tariff (Not estimated)'}]}],
  'observation': [{'id': 'TIME_PERIOD',
    'nam

In [249]:
df = pd.json_normalize(data['structure'])

In [250]:
df

,name,description,annotations,dimensions.dataset,dimensions.series,dimensions.observation,attributes.dataset,attributes.series,attributes.observation
0,WITS - UNCTAD TRAINS Tariff Data,Data flow to access WITS - UNCTAD TRAINS Prefe...,[],[],"[{'id': 'FREQ', 'name': 'Freq', 'keyPosition':...","[{'id': 'TIME_PERIOD', 'name': 'Year', 'keyPos...","[{'id': 'DATASOURCE', 'name': 'DataSource', 'r...",[],"[{'id': 'NOMENCODE', 'name': 'NomenCode', 'rol..."
